In [1]:
!pip3 install beautifulsoup4
!pip3 install requests pandas html5lib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 16.2 MB/s eta 0:00:00


In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass

def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    

In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

In [5]:
response = requests.get(static_url)

In [6]:
response.status_code

200

In [7]:
soup = BeautifulSoup(response.text, 'html.parser')

In [8]:
print(soup.title)

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>


In [12]:
tables = pd.read_html(str(soup))

html_tables = []

In [13]:
html_tables = []

In [14]:
table_bs = soup.find('table',attrs={"class": "wikitable"})
table_bs_data = table_bs.tbody.find_all("tr") 

In [15]:
for i in enumerate(table_bs):
    
    html_tables = table_bs.tbody.find_all("tr") 



In [18]:
first_launch_table = html_tables[0]
print(first_launch_table)

<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11"><span class="cite-bracket">[</span>b<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12"><span class="cite-bracket">[</span>c<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>


In [20]:
column_names = []

In [21]:

for th in first_launch_table.find_all('th'):
    #row = cell.find_all('th')
    
    if not (len(extract_column_from_header(th)) == 0):
        column_names.append(extract_column_from_header(th))
print(column_names)
   

 

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


In [22]:
column_names

['Flight No.',
 'Date and time ( )',
 'Launch site',
 'Payload',
 'Payload mass',
 'Orbit',
 'Customer',
 'Launch outcome']

In [23]:
type(column_names)

list

In [56]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

In [57]:
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        #print(rows)
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            # Flight Number value
            # TODO: Append the flight_number into launch_dict with key `Flight No.`
            launch_dict['Flight No.'] = flight_number
            #print(launch_dict['Flight No.'])
            #print(flight_number)
            datatimelist=date_time(row[0])
            
            # Date value
            # TODO: Append the date into launch_dict with key `Date`
            date = datatimelist[0].strip(',')
            launch_dict['Date'] = date
            #print(launch_dict['Date'])
            
            # Time value
            # TODO: Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            launch_dict['Time'] = time
            #print(time)
              
            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            launch_dict['Version Booster'] = bv
            #print(bv)
            
            # Launch Site
            # TODO: Append the bv into launch_dict with key `Launch Site`
            launch_site = row[2].a.string
            launch_dict['Launch site'] = launch_site
            
            #print(launch_site)
            
            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            launch_dict['Payload'] = payload
            #print(payload)
            
            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'] = payload_mass
            #print(payload_mass)
            
            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            launch_dict['Orbit'] = orbit
            #print(orbit)
            
            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`
            #print(type(row[6].a.string))
            if (row[6].a.string is None):
                row[6].a.string = str({None: 'AAA'}.get(row[6].a.string, row[6].a.string))
                #row[6].a.string = "AAA"
                print(type(row[6].a.string))
            
            #print(row[6].a.string)
            customer = row[6].a.string
            #print(customer)
            launch_dict['Customer'] = customer
            
            
            
            #print(customer)
            
            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome'] = launch_outcome
            
            
            # Booster landing
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing']=booster_landing
            #df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })
    print(launch_dict)
    df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })        



    

{'Flight No.': '7', 'Launch site': 'CCAFS', 'Payload': 'SES-8', 'Payload mass': '3,170 kg', 'Orbit': 'GTO', 'Customer': 'SES', 'Launch outcome': 'Success', 'Version Booster': 'F9 v1.1', 'Booster landing': 'No attempt', 'Date': '3 December 2013', 'Time': '22:41'}
{'Flight No.': '13', 'Launch site': 'Cape Canaveral', 'Payload': 'SpaceX CRS-4', 'Payload mass': '2,216 kg', 'Orbit': 'LEO', 'Customer': 'NASA', 'Launch outcome': 'Success', 'Version Booster': 'F9 v1.1[', 'Booster landing': 'Uncontrolled', 'Date': '21 September 2014', 'Time': '05:52'}
{'Flight No.': '20', 'Launch site': 'Cape Canaveral', 'Payload': 'Orbcomm-OG2', 'Payload mass': '2,034 kg', 'Orbit': 'LEO', 'Customer': 'Orbcomm', 'Launch outcome': 'Success\n', 'Version Booster': 'F9 FT[', 'Booster landing': 'Success', 'Date': '22 December 2015', 'Time': '01:29'}
{'Flight No.': '28', 'Launch site': 'Cape Canaveral', 'Payload': 'JCSAT-16', 'Payload mass': '4,600 kg', 'Orbit': 'GTO', 'Customer': 'SKY Perfect JSAT Group', 'Launch ou

AttributeError: 'NoneType' object has no attribute 'string'

In [42]:
launch_dict

{'Flight No.': '121',
 'Launch site': 'CCSFS',
 'Payload': 'SXM-8',
 'Payload mass': '7,000 kg',
 'Orbit': 'GTO',
 'Customer': None,
 'Launch outcome': 'Success\n',
 'Version Booster': 'F9 B5',
 'Booster landing': 'Success',
 'Date': '6 June 2021',
 'Time': '04:26'}

In [41]:
launch_dict['Customer']

In [108]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

In [108]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

In [109]:
df.to_csv('spacex_web_scraped.csv', index=False)

In [54]:
df

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,121,CCSFS,SXM-8,"7,000 kg",GTO,NaN,Success\n,F9 B5,Success,6 June 2021,04:26
